<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/DistillationNRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.ipynb
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [2]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox
    from che import Props

import numpy as np
from scipy.optimize import root

In [3]:
p=Props(['Ethanol','Water'])

In [4]:
d=DotDict()

d.P = 1e5
d.N = 5
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.2,0.8])
d.FT =320.
d.D = 0.25*d.F
d.B = d.F - d.D

d.F_stage = d.N // 2

d.reflux_ratio = 10.


In [5]:
def boiling_points(P):
    return root(lambda T: np.diagonal(p.Pvap(T)) - P, p.Tb).x

In [13]:
d.bp = boiling_points(d.P)
bp_scaled = (d.bp - np.mean(d.bp))/1.2 + np.mean(d.bp)
bp_minmax = np.min(d.bp), np.max(d.bp)

d.T = RangeArray(np.linspace(*bp_scaled,d.N), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)

Lguess = np.repeat(d.D*(d.reflux_ratio),d.N)
Lguess[d.F_stage:]+=d.F

d.L = RangeArray(Lguess, 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*(d.reflux_ratio+1),d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * d.reflux_ratio* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [14]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [15]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx * p.NRTL_gamma(d.Lx,d.T)* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]* p.NRTL_gamma(d.Vy[0],d.TC) * p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:])
    r.EB[d.F_stage-1] += p.Hl(d.F*d.Fz, d.FT)
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [16]:
eqs_wrapped = wrap(eqs)

In [17]:
sol=root(eqs_wrapped, x, )

In [18]:
res=xtod(sol.x,d2)

In [19]:
eqs(xtod(sol.x,d2))

(array([-9.04580139e+03,  8.09169777e+03, -7.44447673e+02,  3.55207931e+03,
         7.44150322e+02, -7.74880551e+03,  2.90647037e+02, -6.02800244e+02,
        -1.27641252e+04, -6.52246673e+03, -1.51849452e-01,  3.73452887e-01,
         1.18539152e+01, -1.13772568e+01, -1.38181499e+01,  1.33132474e+01,
         5.47849972e+00, -5.61531522e+00, -7.14900951e+00,  7.26009183e+00,
        -4.12861632e+02, -1.13268956e+03, -6.71624095e+02, -1.47559182e+03,
        -8.20970084e+02, -1.69167929e+02, -3.10509618e-03,  4.43934684e+02]),
 {'CONDENSER': array([443.93468351]),
  'EB': array([ -412.86163199, -1132.68955969,  -671.62409504, -1475.59182177,
          -820.97008416]),
  'EQUIL': array([[ -9045.80139306,   8091.69777141],
         [  -744.44767311,   3552.07931201],
         [   744.15032218,  -7748.80550961],
         [   290.64703678,   -602.80024435],
         [-12764.12518298,  -6522.46673258]]),
  'Lin': array([[200.60003339,  49.48996544],
         [158.70565556,  89.87095158],
 